In [18]:
import sys

import numpy as np
from matplotlib import pyplot as plt

from keras.layers import Dense, Input
from keras.models import Model

import energyflow as ef
import energyflow.archs

import omnifold as of
import ibu
import os
import tensorflow as tf
import pandas as pd
import modplot

import uproot

from sklearn.model_selection import train_test_split

In [19]:
tf.device("/cpu:0")

In [20]:
obs_multifold = ['JetPt', 'Z', 'DeltaR']
# obs_multifold = ['P1']
obs = {}
[obs.setdefault(x, {}).update({'func': lambda dset: dset.to_numpy()}) for x in obs_multifold]
obs.items()

dict_items([('JetPt', {'func': <function <listcomp>.<lambda> at 0x59c8416c0>}), ('Z', {'func': <function <listcomp>.<lambda> at 0x59c841260>}), ('DeltaR', {'func': <function <listcomp>.<lambda> at 0x59c8414e0>})])

In [21]:
MuTree = uproot.open("MultiFoldInputFile.root:MuTreeForMultiFold")

In [22]:
MuTree.num_entries

9916863

In [23]:
MuTreeData = MuTree.arrays(["MCJetPt", "MCZ", "MCDeltaR", "RecoJetPt", "RecoZ", "RecoDeltaR"], library="pd")

In [24]:
MuTreeData.drop(MuTreeData.tail(9500000).index,
        inplace = True)

In [25]:
Train, Test = train_test_split(MuTreeData, test_size=0.2)

In [26]:
Train['MC'+'JetPt'].to_numpy()

array([1.3552246, 1.4116211, 4.1035156, ..., 7.6445312, 3.7827148,
       6.961914 ], dtype=float32)

In [27]:
for i, (obkey, ob) in enumerate(obs.items()):
    ob['genobs'], ob['simobs'] = ob['func'](Train['MC'+obkey]), ob['func'](Train['Reco'+obkey])
    ob['truobs'], ob['dataobs'] = ob['func'](Test['MC'+obkey]), ob['func'](Test['Reco'+obkey])
    print('Done with', obkey)

Done with JetPt
Done with Z
Done with DeltaR


In [28]:
ob['genobs']

array([0.00019148, 0.00032717, 0.08324414, ..., 0.16178203, 0.0076366 ,
       0.01538287], dtype=float32)

In [29]:
# set up the array of data/simulation detector-level observables
X_det = np.asarray([np.concatenate(
    (obs[obkey]['dataobs'], obs[obkey]['simobs'])) for obkey in obs_multifold]).T
Y_det = pd.get_dummies(np.concatenate((np.ones(len(obs['JetPt']['dataobs'])),
                                                np.zeros(len(obs['JetPt']['simobs'])))), dtype=int).to_numpy()

In [30]:
# set up the array of generation particle-level observables
X_gen = np.asarray([np.concatenate(
    (obs[obkey]['genobs'], obs[obkey]['genobs'])) for obkey in obs_multifold]).T
Y_gen = pd.get_dummies(np.concatenate((np.ones(len(obs['JetPt']['genobs'])),
                                                np.zeros(len(obs['JetPt']['genobs'])))), dtype=int).to_numpy()

In [31]:
# standardize the inputs
X_det = (X_det - np.mean(X_det, axis=0))/np.std(X_det, axis=0)
X_gen = (X_gen - np.mean(X_gen, axis=0))/np.std(X_gen, axis=0)

In [32]:
# Specify the training parameters
# model parameters for the Step 1 network
model_layer_sizes = [100, 100]  # use this for the full network size
# model_layer_sizes = [100, 100, 100]  # use this for the full network size
det_args = {'input_dim': len(obs_multifold), 'dense_sizes': model_layer_sizes,
            'patience': 10, 'filepath': 'Step1_{}', 'save_weights_only': False,
            'modelcheck_opts': {'save_best_only': True, 'verbose': 1}}
# model parameters for the Step 2 network
mc_args = {'input_dim': len(obs_multifold), 'dense_sizes': model_layer_sizes,
           'patience': 10, 'filepath': 'Step2_{}', 'save_weights_only': False,
           'modelcheck_opts': {'save_best_only': True, 'verbose': 1}}
# general training parameters
fitargs = {'batch_size': 2000, 'epochs': 2, 'verbose': 1}
# reweight the sim and data to have the same total weight to begin with
ndata, nsim = np.count_nonzero(Y_det[:,1]), np.count_nonzero(Y_det[:,0])
wdata = np.ones(ndata)
winit = ndata/nsim*np.ones(nsim)

In [ ]:
# apply the OmniFold procedure to get weights for the generation
multifold_ws = of.omnifold(X_gen, Y_gen, X_det, Y_det, wdata, winit,
                                (ef.archs.DNN, det_args), (ef.archs.DNN, mc_args),
                                fitargs, val=0.2, it=3, trw_ind=-2, weights_filename='Test')

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 3)]               0         
                                                                 
 dense_0 (Dense)             (None, 100)               400       
                                                                 
 activation_6 (Activation)   (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 activation_7 (Activation)   (None, 100)               0         
                                                                 
 output (Dense)              (None, 2)                 202       
                                                                 
 activation_8 (Activation)   (None, 2)                 0   

In [ ]:
# additional histogram and plot style information
hist_style = {'histtype': 'step', 'lw': 1, 'zorder': 2}
gen_style = {'linestyle': '--', 'color': 'blue', 'lw': 1.15, 'label': 'Gen.'}
truth_style = {'step': 'mid', 'edgecolor': 'green', 'facecolor': (0.75, 0.875, 0.75),
               'lw': 1.25, 'zorder': 0, 'label': '``Truth\"'}
ibu_style = {'ls': '-', 'marker': 'o', 'ms': 2.5, 'color': 'gray', 'zorder': 1}
omnifold_style = {'histtype': 'step', 'lw': 2, 'zorder': 2}

In [ ]:
plt.rcParams['figure.figsize'] = (12,4)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.family'] = 'serif'

In [ ]:
fig, axs = plt.subplots(1,3)
bindef = [np.linspace(-3,3,25), np.linspace(-3,3,75), np.linspace(-3,3,75)]
mcbindef = [np.linspace(-3,3,25), np.linspace(0,1,25), np.linspace(0,1,25)]

for i,(obkey,ob) in enumerate(obs.items()):
    _,_,_=axs[i].hist(ob['dataobs'], bins=bindef[i], color='red', histtype='step', label='``Data\"')
    axs[i].hist(ob['simobs'], bins=bindef[i], color='orange', label='Sim.', histtype='step')
    _,_,_=axs[i].hist(ob['genobs'], bins=bindef[i], color='green', label='Gen.', histtype='step', ls=':', lw=2)
    _,_,_=axs[i].hist(ob['truobs'], bins=bindef[i], color='blue', label='True', alpha=0.5)
    _,_,_=axs[i].hist(ob['genobs'], bins=bindef[i], weights=multifold_ws[6], color='black', label='Multifolded', **omnifold_style)
    axs[i].set_yscale("log")
    axs[i].legend(frameon=False)
